## Sentiment Analysis - Prediction Execution
Based on model build on another code,
build specifically for cekmedsos.com, communicating through database layer

In [ ]:
# installations
!pip install pymysql
!pip install pandas
!pip install uuid
!pip install time
!pip install os
!pip install string
!pip install re
!pip install tensorflow

In [6]:
# Imports
import pymysql.cursors
import pandas as pd
import numpy as np
import uuid
import time
import os
import string
import re
import tensorflow as tf


from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from datetime import datetime
from sklearn.model_selection import train_test_split

## need to install this !!!!
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Activation, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow import keras

# database connection properties
db_host='202.157.185.40'
db_user ='cekmedsos_db'
db_password='282E~f0si'
db_database='cekmedsos_database'

# save model to what filename?
model_data_filename = 'train-data-01.h5'

# training data file
train_data_file = './train-data/train-data-labeled.csv'

# model parameters
num_of_epoch = 128
max_kata = 100

print('Finished loading library')

Finished loading library


### Retrieving data jobs from database
For jenis_analisa = 2, queuing based on FIFO

In [7]:
# Connect to the database
connection = pymysql.connect(host=db_host,
                             user=db_user,
                             password=db_password,
                             database=db_database,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

# get available jobs from database server, first come first serve
# sql = "select id, hastag, `parameter` from screen_analisis_ai where active = 1 and status = 1 and jenis_analisa = 1 order by created asc, id asc limit 1"
sql = "select id, hastag, `parameter` \
from screen_analisis_ai \
where active = 1 \
and status = 1 \
and jenis_analisa = 3 \
order by created asc, id asc limit 1"

row_count = cursor.execute(sql)

if(row_count == 0):
    # get out, nothing to do
    print('Zero jobs, quitting now')
    quit()

result = cursor.fetchall()
database_keyword_id = result[0]['hastag']
similarity_treshold = result[0]['parameter']
i_process_id = result[0]['id']
screen_name = ''

print(database_keyword_id)
print(similarity_treshold)
print(i_process_id)
print('Finished querying jobs')

Zero jobs, quitting now


IndexError: tuple index out of range

### Marking this jobs as run

In [ ]:
# Connect to the database
connection = pymysql.connect(host='202.157.176.225',
                             user='cekmedsos_db',
                             password='kuku838485*#',
                             database='cekmedsos_database',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
cursor = connection.cursor()

# Prepare SQL Statement
print(i_process_id)
sql = "update screen_analisis_ai set status = 2, last_status_update = now(), start_process = now() where id = %s"

# execute
cursor.execute(sql, i_process_id)

# commit record
connection.commit()

print('Job mark as run')

### Inserting Job Header Data

In [ ]:
# Connect to the database
connection = pymysql.connect(host='202.157.176.225',
                             user='cekmedsos_db',
                             password='kuku838485*#',
                             database='cekmedsos_database',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

#
# Create Header Record
sql = "insert into ret_analysis_header (job_id, datetime_start, user_id) values (%s, %s, %s)"
# Execute the query
print('ProcessID: ' + str( i_process_id))
cursor.execute(sql, (str(i_process_id), datetime.now(), 1 ))

#
# Create Parameter Record
sql = "insert into ret_analysis_parameter (job_id, param_id, param_name, param_value) values (%s, %s, %s, %s)"
# Execute the query
cursor.execute(sql, (i_process_id, 1, 'Model ID', model_data_filename))
cursor.execute(sql, (i_process_id, 2, 'DB_ID', database_keyword_id))

# Commit Record
connection.commit()

print('Job Header Data Inserted')

### Querying tweet data from database based on jobs id


In [ ]:
#
# Query to get tweet data, apply analitics to this dataset
#

# Connect to the database
connection = pymysql.connect(host='202.157.176.225',
                             user='cekmedsos_db',
                             password='kuku838485*#',
                             database='cekmedsos_database',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

s_query_string = 'select id, tweet, tweet_date_time from ret_tweet where '
# s_query_string = 'select id, tweet, tweet_date_time from vw_ret_tweet_clean where '

if (screen_name != ''):
    # print('use screen name')
    s_query_string = s_query_string + 'screen_name = "' + screen_name + '" and db_id = "' + str(database_keyword_id) + '"'
else:
    # print('no use')
    s_query_string = s_query_string + 'db_id = "' + database_keyword_id.replace('"','') + '"'
    
print(s_query_string)
df = pd.read_sql(s_query_string, con=connection)

# Close Connection
connection.close()

# see result
df.head()

### Text pre-processing functions
1. Case Folding
2. Stemming
3. Tokenizer

In [ ]:
# function stemming
import nltk
from sklearn.pipeline import Pipeline
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# nltk.download('stopwords')

def stemming(comment):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in comment:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    return d_clean
    
# function case folding
import re
def casefolding(comment):
    comment = comment.lower()
    comment = comment.strip(" ")
    comment = re.sub(r'[?|$|.|!_:")(-+,]','',comment)
    return comment

def clean_up_tag(comment):
    x_ret = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",comment).split())
    return x_ret

# Text Preprocessing, 
def text_preproc(strIn):
    # case folding
    strOut = strIn.lower()
    
    # remove numbers
    strOut = re.sub(r"\d+", "", strOut)
    
    # remote punctuation
    strOut = strOut.translate(str.maketrans("","",string.punctuation))
    
    # remove whitspace
    strOut = strOut.strip()
    
    # 
    strOut = re.sub('\s+',' ',strOut)
    return strOut

print('functions is build')

### Prepared data from database to predict


In [ ]:
# perform stemming on 'sw'
# df['stemmed'] = df['tokenized_tweet'].apply(stemming)
# x = stemming(['Sistem Merit Didefinisikan Sebagai Kebijakan dan Manajemen ASN Yang Berdasarkan Pada Kualifikasi, Kompetensi dan Kinerja, Yang Diberlakukan Secara Adil dan Wajar Dengan Tanpa Diskriminasi.  #PerkuatPemulihanEkonomi  https://t.co/8al7sOKFZU'])
from tqdm import tqdm
pd.options.mode.chained_assignment = None 

# remove first
df['tweet'] = df['tweet'].apply(clean_up_tag)

# skip stemming
df['stemmed'] = df['tweet']
# text = [0]
# with tqdm(total=len(df)) as pbar:
#     for i in range(0, len(df)):
#         text[0] = df['tweet'][i]
#         x = stemming(text)
#         df['stemmed'][i] = x
        
#         pbar.update(1)

# pbar.close()
df.head(100)

In [ ]:
df['stemmed'] = df['stemmed'].apply(casefolding)
df['stemmed'] = df['stemmed'].apply(text_preproc)

sw = stopwords.words('indonesian')

#tokenized
df['tokenized_tweet'] = df.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

# apply stopword removal
df['tokenized_tweet'] = df.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)

df.head(100)

### Load model and prepare encoding calculation

In [ ]:
# load data
df_train = pd.read_csv(train_data_file,sep="\t",)
df_train.head()

# array of text, tweet content
text = df_train['Tweet'].tolist()

token = Tokenizer()
token.fit_on_texts(text)

def get_encode(x):
    x = token.texts_to_sequences(x)
    x = pad_sequences(x, maxlen = max_kata, padding='post')
    return x

# loading and testing models
new_model = tf.keras.models.load_model(model_data_filename)

# display summary
new_model.summary()

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

# prepare new column to hold prediction value
df['sa'] = ''

#for i in range(0, len(df)):

x = ['']
for i in range(0, len(df)):
# for i in range(0, 10):
    x[0] = ' '.join(df['tokenized_tweet'][i])
    y = get_encode(x)
    sa = np.argmax(new_model.predict(y), axis=-1)
    
    # convert negative values
    if(sa[0] == 2):
        sa[0] = -1
        
    df['sa'][i] = sa[0]

# print('Finished predicting')
df.head(50)


### Writeback to database
Database structure:
- job_id
- sentiment_value
- tweet_id
- 

Notes on return sentiment values
- 1 >> positif >> stay
- 2 >> negatif >> convert to -1
- 0 >> netral >> stay

In [ ]:
# Connect to the database
connection = pymysql.connect(host='202.157.176.225',
                             user='cekmedsos_db',
                             password='kuku838485*#',
                             database='cekmedsos_database',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

#
# Create Tweet Cluster Record
sql = "insert into ret_sentiment_result (job_id, tweet_id, sentiment_class) values (%s, %s, %s)"
                 
# inserting tweet cluster
for i in range(0, len(df)):
    
    # Execute the query
    cursor.execute(sql, (i_process_id, df['id'][i], df['sa'][i]))
    
connection.commit()
connection.close()

print('finished inserting sentiment data')

### Record Processing Time

In [ ]:
# Connect to the database
connection = pymysql.connect(host='202.157.176.225',
                             user='cekmedsos_db',
                             password='kuku838485*#',
                             database='cekmedsos_database',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

#
# Create Parameter Record
sql = "insert into ret_analysis_parameter (job_id, param_id, param_name, param_value) values (%s, %s, %s, %s)"
# Execute the query
cursor.execute(sql, (i_process_id, 3, '#Tweet Processed',len(df)))

#
# Create Tweet Cluster Record
sql = "update ret_analysis_header set datetime_finish = %s where job_id = %s"
# Executing query
cursor.execute(sql, (datetime.now(),i_process_id) )

print(i_process_id)

sql = "update screen_analisis_ai set status = 3, end_process = now(), duration = TIMESTAMPDIFF(second,start_process, end_process) where id = %s"
# Executing query
cursor.execute(sql,i_process_id)

connection.commit()
connection.close()

print('job finished')